# A SCRUD API 
A SCRUD API collects contexts, schemas, etc. and generates top-level discovery documents.

In [9]:
import traitlets as T
import traitlets.config
from tornado import web
from notebook.base.handlers import IPythonHandler
from notebook.utils import url_path_join as ujoin

In [15]:
with __import__("importnb").Notebook():
    from ScrudHandler import ScrudHandler

In [13]:
DOC_MIME = "application/ld+json"
DOC_REL = "http://www.w3.org/ns/hydra/core#apiDocumentation"

In [14]:
context = {
    "hydra": "http://w3.org/ns/hydra/core#"
}

In [11]:
class ScrudAPIDocHandler(ScrudHandler):
    async def value(self):
        return self.scrud.api_doc()

In [12]:
class ScrudAPI(T.config.LoggingConfigurable):
    full_url = T.Unicode()
    base_url = T.Unicode()
    tornado_app = T.Any()
    prefix = T.Unicode("scrud")
    title = T.Unicode("SCRUD API")
    description = T.Unicode("A reference implementation of a SCRUD-ful API")
    supported_classes = T.Dict()
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        prefix = ujoin(self.prefix, "doc")
        url = ujoin(self.base_url, prefix) + "/?"
        self.tornado_app.add_handlers(".*", [(
            url, 
            ScrudAPIDocHandler, 
            dict(scrud=self, prefix="doc")
        )])
    
    def headers(self):
        return [(
            "Link", 
            f'''<{self.doc_url}>; rel="{DOC_REL}"; type="{DOC_MIME}"'''
        )]
    
    @property
    def api_base(self):
        return ujoin(self.base_url, self.prefix)
    
    @property
    def api_url(self):
        return ujoin(self.full_url, self.prefix)
    
    @property
    def doc_url(self):
        return ujoin(self.api_url, "doc")
    
    def supports_operation(self, rdf_class, *operations):
        if rdf_class not in self.supported_classes:
            self.supported_classes[rdf_class] = {
                "@id": rdf_class,
                "supportedOperation": []
            }
        self.supported_classes[rdf_class]["supportedOperation"].append(*operations)
    
    def api_doc(self):
        return {
            "@type": "ApiDocumentation",
            "@id": self.doc_url,
            "supportedClass": list(self.supported_classes.values())
        }
    
    def add_handler(self, default_handlers):
        self.tornado_app.add_handlers(".*", default_handlers(self))